In [1]:
from flask import Flask, render_template, request, redirect, url_for, session
import mysql.connector
import hashlib
import os

In [6]:
app = Flask(__name__)
app.secret_key = os.urandom(24)  # Generate a random secret key

db = mysql.connector.connect(
    host="localhost", user="root", password="india123", database="appy"
)
cursor = db.cursor()

# Function to hash passwords using SHA-256
# def hash_password(password):
#     return hashlib.sha256(password.encode('utf-8')).hexdigest()

# Function to check hashed password
def check_password(hashed_password, user_password):
    return hashed_password == user_password

@app.route("/")
def index():
    if 'username' not in session:
        return redirect(url_for('login'))
    username = session['username']
    cursor.execute("SELECT * FROM Train")
    trains = cursor.fetchall()
    return render_template("index.html", username=username, trains=trains)

In [7]:
check_password('india123', 'india123')

True

In [ ]:
@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']

        # Check if admin
        cursor.execute("SELECT * FROM Admin WHERE Username = %s", (username,))
        admin = cursor.fetchone()
        if admin:
            if check_password(admin[2], password):
                session['username'] = username
                return redirect(url_for('index'))
            else:
                error = "Incorrect password. Please try again."
                return render_template("login.html", error=error)

        # Check if user
        cursor.execute("SELECT * FROM User WHERE Username = %s", (username,))
        user = cursor.fetchone()
        if user:
            if check_password(user[2], password):
                session['username'] = username
                return redirect(url_for('index'))
            else:
                error = "Incorrect password. Please try again."
                return render_template("login.html", error=error)

        # Username not found
        error = "Invalid username. Please check your username and try again."
        return render_template("login.html", error=error)
    
    return render_template("login.html")